# Кластеризация и SVM

## Ф.И.О: _________

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import adjusted_rand_score, accuracy_score
from sklearn.datasets import load_iris, make_classification, make_moons
from cvxopt import matrix, solvers

### Task 1: Реализация алгортим K-Means (6 points)

#### 1.1. Реализуйте алгоритм K-means без использования библиотек, реализующих данный метод (например, библиотеки sklearn) (3 points)

In [ ]:
def k_means(X, k, max_iters=3, tol=1e-4):
    """ Реализуйте алгоритм K-Means """
    np.random.seed(42)
    # Your code here
    pass

# Automatic Checker
def check_k_means():
    iris = load_iris()
    X = iris.data
    y_true = iris.target

    labels_custom, centroids_custom = k_means(X, k=3)
    kmeans_sklearn = KMeans(n_clusters=3, random_state=42, n_init=10).fit(X)
    labels_sklearn = kmeans_sklearn.labels_

    score_custom = adjusted_rand_score(y_true, labels_custom)
    score_sklearn = adjusted_rand_score(y_true, labels_sklearn)
    print(score_custom)
    print(score_sklearn)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(X[:, 0], X[:, 1], c=labels_custom, cmap='viridis', marker='o')
    plt.scatter(centroids_custom[:, 0], centroids_custom[:, 1], s=300, c='red', marker='X')
    plt.title('K-Means Clustering')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

    assert labels_custom is not None, "Function should return labels."
    assert len(labels_custom) == len(X), "Should return labels for all points."
    assert score_custom > 0.7, "Clustering should achieve a higher score."
    print(f"Task 1.1 Passed! Adjusted Rand Score: {score_custom:.2f}")

In [ ]:
check_k_means()

#### 1.2. Реализуйте алгоритм K-means++ без использования библиотек, реализующих данный метод (например, библиотеки sklearn) (3 points)

In [ ]:
def initialize_plus_plus(X, k):
    """ Инициализируйте центры кластеров """
    np.random.seed(42)
    # Your code here
    pass

def k_means_plus_plus(X, k, max_iters=100, tol=1e-4):
    """ Реализуйте алгоритм K-means++ """
    centroids = initialize_plus_plus(X, k)
    # Your code here
    pass

# Automatic Checker
def check_k_means_plus_plus():
    iris = load_iris()
    X = iris.data
    y_true = iris.target

    labels_custom, centroids_custom = k_means_plus_plus(X, k=3)
    kmeans_sklearn = KMeans(n_clusters=3, init='k-means++', random_state=42, n_init=10).fit(X)
    labels_sklearn = kmeans_sklearn.labels_

    score_custom = adjusted_rand_score(y_true, labels_custom)
    score_sklearn = adjusted_rand_score(y_true, labels_sklearn)
    print(score_custom)
    print(score_sklearn)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(X[:, 0], X[:, 1], c=labels_custom, cmap='viridis', marker='o')
    plt.scatter(centroids_custom[:, 0], centroids_custom[:, 1], s=300, c='red', marker='X')
    plt.title('K-Means++ Clustering')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

    assert labels_custom is not None, "Function should return labels."
    assert len(labels_custom) == len(X), "Should return labels for all points."
    assert score_custom > 0.7, "Clustering should achieve a higher score."
    print(f"Task 1.2 Passed! Adjusted Rand Score: {score_custom:.2f}")


In [ ]:
check_k_means_plus_plus()

### Task 2: Оценка алгоритма DBSCAN (4 points)

Вы анализируете движение такси в городе, чтобы определить популярные зоны посадки. У Вас есть набор данных, представляющий места посадки такси.

```
data_points = [
    (1.2, 3.1), (2.5, 3.3), (1.8, 2.9), (8.2, 7.5),
    (8.1, 7.7), (25.3, 80.2), (2.0, 3.0), (3.0, 3.5),
    (1.1, 1.2), (90.0, 91.2), (1.5, 3.2), (8.0, 7.6),
    (2.2, 3.1), (7.9, 7.8), (1.3, 3.0)
]
```

Каждая точка представляет посадку такси с координатами (x, y) в километрах от центра города.

С помощью алгортима DBSCAN сгруппируйте эти поездки на такси, чтобы определить зоны с высокой плотностью посадки (Выберите максимальное расстояние между точками = 1.5 и минимальное число точек в группе = 3), а какие зоны посадки являются случайными выбросами.

Для решения этой задачи не используйте библиотеки с реализацией DBSCAN (например, библиотеки sklearn)

In [ ]:
# Your code here

### Task 3 Оценка алгоритма SVM (4 points)

Рассмотрим набор данных в 2D-пространстве, где первые два значения - это координаты точек, а третье значение - метка класса:

```
data_points = [
    (-4, 2, 1), (-1, -1, -1), (0, -2, -1), (1, -2, -1),
    (2, 3, 1), (3, 3, 1), (4, 4, 1), (-4, -3, -1), (-5, -2, -1),
    (5, 2, 1), (6, 1, 1), (-6, -1, -1), (-7, -3, -1), (7, 3, 1),
    (8, 4, 1), (-8, -5, -1), (9, 5, 1), (-9, -4, -1), (10, 6, 1),
]
```

1. Определите уравнение прямой, которая наилучшим образом разделяет два класса.
2. Определите величину зазора (margin) и вектор, ортогональный разделяющей границе
3. Определите, какие точки являются опорными векторами.


In [ ]:
# Your code here

### Task 4 Реализация алгоритма SVM (6 points)

#### 4.1. Реализуйте алгоритм Linear SVM без использования библиотек, реализующих данный метод (например, библиотеки sklearn) (3 points)

In [ ]:
def linear_svm(X, y, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
    """ Реализуйте Linear SVM """
    np.random.seed(42)
    # Your code here
    pass

def predict_svm(X, weights, bias):
    return np.sign(np.dot(X, weights) - bias)

# Automatic Checker
def check_linear_svm():
    X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=42)
    y = np.where(y == 0, -1, 1)

    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    weights, bias = linear_svm(X, y)
    y_pred = predict_svm(X, weights, bias)

    svm_sklearn = SVC(kernel='linear', random_state=42)
    svm_sklearn.fit(X, y)
    y_pred_sklearn = svm_sklearn.predict(X)

    accuracy = accuracy_score(y, y_pred)
    accuracy_sklearn = accuracy_score(y, y_pred_sklearn)

    assert accuracy > 0.85, "Accuracy should be higher"
    print(f"Task 4.1 Passed! Accuracy: {accuracy:.2f}")

In [ ]:
check_linear_svm()

#### Task 4.2: Реализуйте Kernel SVM с использованием RBF ядра без использования библиотек, реализующих данный метод (например, библиотеки sklearn) (3 points)

In [ ]:
def rbf_kernel(X, Y=None, gamma=0.1):
    if Y is None:
        Y = X
    """ Определите RBF kernel """
    # Your code here
    pass

def kernel_svm(X, y, gamma=0.1, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
    """ Реализуйте Kernel SVM """
    np.random.seed(42)
    # Your code here
    pass

def predict_kernel_svm(X, X_train, y_train, alpha, gamma=0.1):
    K = rbf_kernel(X, X_train, gamma)
    return np.sign(np.dot(K, alpha * y_train))

# Automatic Checker
def check_kernel_svm():
    X, y = make_moons(n_samples=50, noise=0.01, random_state=42)
    y = np.where(y == 0, -1, 1)

    alpha = kernel_svm(X, y)
    y_pred = predict_kernel_svm(X, X, y, alpha)

    svm_sklearn = SVC(kernel='rbf', gamma=0.1, random_state=42)
    svm_sklearn.fit(X, y)
    y_pred_sklearn = svm_sklearn.predict(X)

    accuracy = accuracy_score(y, y_pred)
    accuracy_sklearn = accuracy_score(y, y_pred_sklearn)

    print(accuracy_sklearn)

    assert accuracy > 0.8, "Accuracy should be higher"
    print(f"Task 4.2 Passed! Accuracy: {accuracy:.2f}")

In [ ]:
check_kernel_svm()